# Challenge: Circuit Extraction

Our normal simplification procedure always yields ZX-diagrams that can be turned back into (ancilla-free) circuits.

**Challenge**: find a means of deterministically realising the ZX-diagram g2 below, _without increasing T-count_.

In [1]:
import random, math, os, time
import sys; sys.path.append('..')
import pyzx as zx
from copy import deepcopy
#%config InlineBackend.figure_format = 'svg'
from extract_ancilla import *
from extract_helpers import *
from graph_helpers import *
from lcomping import *
from phase_gadgets import *

Here, we load the circuit and do our "normal" simplification, which reduces T-count from 21 to 15.

In [2]:
c = zx.Circuit.load('./circuits/tof_3.qasm').to_basic_gates()
g = c.to_graph()
g1 = g.copy()
zx.simplify.full_reduce(g1)
zx.draw(g1)
print("old T-count: %s --> new T-count: %s" % (zx.tcount(g), zx.tcount(g1)))

old T-count: 21 --> new T-count: 15


...and indeed the resulting ZX-diagram can be transformed back into a circuit.

In [3]:
c1 = zx.extract_circuit(g1.copy())
c1 = zx.optimize.basic_optimization(c1.to_basic_gates()).to_basic_gates()
zx.draw(c1.to_graph())
print(c1.stats())

Circuit  on 5 qubits with 53 gates.
        15 is the T-count
        38 Cliffords among which
        29 2-qubit gates (24 CNOT, 5 other) and
        6 Hadamard gates.


At this point, we can use the simplification rule **gadget_phasepoly** to reduce the T-count by _one more_, down to 14.

In [4]:
g2 = g1.copy()

while True:
    m = zx.rules.match_gadgets_phasepoly(g2)
    if len(m) != 0: zx.rules.apply_gadget_phasepoly(g2, m)
    else: break
zx.draw(g2)
print("old T-count: %s --> new T-count: %s" % (zx.tcount(g1), zx.tcount(g2)))

old T-count: 15 --> new T-count: 14


...but the circuit no longer extracts!

In [5]:
g3 = g2.copy()
g4 = g3.copy()
zx.simplify.full_reduce(g4)
g4.normalize()
print(find_max_delayed_flow(OpenGraph(g4), continue_on_failure=True, quiet=False))
g4.add_edge((41,43), EdgeType.HADAMARD)
highlight_missing_correction_set(g4)
g4.remove_vertex(38)
print(find_max_delayed_flow(OpenGraph(g4), continue_on_failure=True, quiet=False))
highlight_missing_correction_set(g4)
#zx.draw(g3)
#c2 = zx.extract_circuit(deepcopy(g3))

Unsolvable; choices for lacking vert: {38, 39, 41, 12, 45, 14, 17, 18}
({5: {48}, 7: {47}, 13: {49}, 15: {49, 50, 15}, 16: {50}, 37: {51}, 43: {48, 43, 47}, 38: {-1}, 39: {39}, 41: {41}, 45: {45}, 12: {12, 38}, 14: {38, 14}, 17: {17, 38}, 18: {38}}, {47: 0, 48: 0, 49: 0, 50: 0, 51: 0, 5: 1, 7: 1, 13: 1, 15: 1, 16: 1, 37: 1, 43: 1, 38: 2, 39: 3, 41: 3, 45: 3, 12: 4, 14: 4, 17: 4, 18: 4})


Unsolvable; choices for lacking vert: {41, 43, 12, 14, 17, 18}
Unsolvable; choices for lacking vert: {17, 18, 12, 14}
({5: {48}, 7: {47}, 13: {49}, 15: {49, 50, 15}, 16: {50}, 37: {51}, 39: {48, 39}, 45: {48, 45, 47}, 41: {-1}, 43: {43}, 17: {-1}, 12: {17, 12}, 14: {17, 14}, 18: {17}}, {47: 0, 48: 0, 49: 0, 50: 0, 51: 0, 5: 1, 7: 1, 13: 1, 15: 1, 16: 1, 37: 1, 39: 1, 45: 1, 41: 2, 43: 3, 17: 4, 12: 5, 14: 5, 18: 5})


Here is the ZX-diagram. **Can you extract it?**

In [6]:
ac = extract_circuit_with_ancillae(deepcopy(g3), quiet = True)
ac.do_basic_optimization()
zx.draw(g3)
zx.draw(ac)
#print(ac.compare_tensors(g3))
print(ac.stats())

Circuit  on 6 qubits with 65 gates.
        14 is the T-count
        51 Cliffords among which
        36 2-qubit gates (31 CNOT, 5 other) and
        7 Hadamard gates.
        1 of the qubits is/are ancillary.


In [7]:
g3 = g2.copy()
g3.normalize()
insert_vertex(g3, (21,26), VertexType.Z, 0)
flip_edge_type(g3, (21,51))
flip_edge_type(g3, (26,51))
insert_vertex(g3, (10,4), VertexType.Z, 0)
flip_edge_type(g3, (4,52))
flip_edge_type(g3, (10,52))
zx.draw(g3, labels=True)

highlight_missing_correction_set(g3)
#g3.set_phase(37, Fraction(2,2))
#g3.set_phase(21, 0)
#pivot_single_edge(g3, (37,21), "hopelijk", quiet = True)
print(find_max_delayed_flow(OpenGraph(g3), continue_on_failure=True))
g3.remove_vertices([37,38])
highlight_missing_correction_set(g3)
g3.remove_vertices([6,32])
highlight_missing_correction_set(g3)
g3.remove_vertices([39,40])
highlight_missing_correction_set(g3)
g3.remove_vertices([29,8])
highlight_missing_correction_set(g3)
g3.remove_vertices([34,9])
highlight_missing_correction_set(g3)
g3.remove_vertices([41,42])
highlight_missing_correction_set(g3)
lcomp_single_vertex(g3, 10, "hopelijk", quiet = True)
zx.simplify.full_reduce(g3)
g3.normalize()
insert_vertex(g3, (21,26), VertexType.Z, 0)
flip_edge_type(g3, (21,57))
flip_edge_type(g3, (26,57))
highlight_missing_correction_set(g3)
g3.remove_vertices([12,28])
highlight_missing_correction_set(g3)
g3.remove_vertices([46])
highlight_missing_correction_set(g3)
print(find_max_delayed_flow(OpenGraph(g3), continue_on_failure=True))
g3.remove_vertices([17,33])
highlight_missing_correction_set(g3)
print(find_max_delayed_flow(OpenGraph(g3), continue_on_failure=True))

({5: {48}, 7: {47}, 11: {48, 11, 51, 47}, 13: {49}, 15: {49, 50, 15}, 16: {50}, 19: {48, 19, 51}, 20: {51, 20, 47}, 21: {51}, 43: {48, 43, 47}, 37: {-1}, 6: {37, 6}, 8: {8, 37}, 9: {9, 37}, 10: {37}, 39: {37, 39}, 41: {41, 37}, 45: {45, 37}, 12: {12, 21}, 14: {21, 14}, 17: {17, 21}, 18: {21}, 52: {10, 21}}, {47: 0, 48: 0, 49: 0, 50: 0, 51: 0, 5: 1, 7: 1, 11: 1, 13: 1, 15: 1, 16: 1, 19: 1, 20: 1, 21: 1, 43: 1, 37: 2, 6: 3, 8: 3, 9: 3, 10: 3, 39: 3, 41: 3, 45: 3, 12: 4, 14: 4, 17: 4, 18: 4, 52: 4})


({5: {54}, 7: {53}, 13: {55}, 15: {56, 15, 55}, 16: {56}, 21: {57}, 43: {43, 53, 54}, 52: {21}, 17: {-1}, 14: {17, 14}, 18: {17}}, {53: 0, 54: 0, 55: 0, 56: 0, 57: 0, 5: 1, 7: 1, 13: 1, 15: 1, 16: 1, 21: 1, 43: 1, 52: 2, 17: 3, 14: 4, 18: 4})


({5: {54}, 7: {53}, 13: {55}, 15: {56, 15, 55}, 16: {56}, 21: {57}, 43: {43, 53, 54}, 52: {21}, 18: {-1}, 14: {14}}, {53: 0, 54: 0, 55: 0, 56: 0, 57: 0, 5: 1, 7: 1, 13: 1, 15: 1, 16: 1, 21: 1, 43: 1, 52: 2, 18: 3, 14: 4})


In [8]:
g3 = Graph()
add_vertex(g3, 0, 0, VertexType.BOUNDARY, None)
add_vertex(g3, 1, 0, VertexType.BOUNDARY, None)
add_vertex(g3, 2, 0, VertexType.BOUNDARY, None)
add_vertex(g3, 0, 2, VertexType.BOUNDARY, None)
add_vertex(g3, 1, 2, VertexType.BOUNDARY, None)
add_vertex(g3, 2, 2, VertexType.BOUNDARY, None)
add_vertex(g3, 1, 1, VertexType.Z, Fraction(1,4))
g3.set_inputs([0,1,2])
g3.set_outputs([3,4,5])
g3.add_edges([(i, 6) for i in range(6)], EdgeType.HADAMARD)
g3.normalize()
highlight_missing_correction_set(g3)

In [9]:
if not 's' in vars():
    s = 0
s += 1
#s = 7 # 1 inp 1 out 2 internal => triangle attached to input that breaks gflow no matter what plane
#s = 13 # mn plaatje
#s = 16 # haal lacking vert [12,13] weg, verder extra unlacked
s = 25
print(f"Attempt {s}")
g4 = completely_random_graph(input_count=5, output_count=5, internal_count=10, p_YZ = 0.33, p_XZ = 0.33, p_edge = 0.25, seed = s)
print(find_max_delayed_flow(OpenGraph(g4), continue_on_failure=True))
highlight_missing_correction_set(g4)
g4.remove_vertices([11,10])
highlight_missing_correction_set(g4)

Attempt 25
({1: {-1}, 3: {-1}, 5: {-1}, 7: {-1}, 9: {-1}, 11: {-1}, 12: {32}, 23: {34, 36, 38, 11, 12}, 13: {-1}, 22: {34, 22}, 26: {34, 26, 38}, 16: {16, 12, 22}, 17: {11, 12, 13, 22}, 19: {19, 12}, 29: {29, 13}}, {32: 0, 34: 0, 36: 0, 38: 0, 30: 0, 1: 1, 3: 2, 5: 3, 7: 4, 9: 5, 11: 6, 12: 7, 23: 8, 13: 9, 22: 10, 26: 10, 16: 11, 17: 11, 19: 11, 29: 11})


In [10]:
ac = extract_circuit_with_ancillae(g4, quiet = True)
ac.do_basic_optimization()
zx.draw(g4)
zx.draw(ac)

Something went wrong unexpectedly:
44
Problem graph:


KeyError: 44

In [11]:
g3 = Graph()
# 0: ZX-input; 1: ZX-output; 2: i; 3: o; 4,5: u,v
[ZXi, ZXo, i, o, u, v] = g3.add_vertices(6)
g3.set_inputs([ZXi])
g3.set_outputs([ZXo])
for w in range(2,6):
    g3.set_type(w, VertexType.Z)
g3.add_edges([(ZXi,i),(ZXo,o),(i,u),(i,v),(u,o),(v,o),(u,v)], EdgeType.HADAMARD)
g3.normalize()
g3.set_position(u, -2, 2.5)
g3.set_position(v, 2, 2.5)
replace_measurement(g3, i, "XY", 0, [i])
replace_measurement(g3, u, "XY", 0, [u])
replace_measurement(g3, v, "XY", 0, [v])
#g3.remove_vertex(v)
highlight_missing_correction_set(g3)

In [14]:
g3 = Graph()
[ZXi, ZXo, i, o, gadget] = g3.add_vertices(5)
g3.set_inputs([ZXi])
g3.set_outputs([ZXo])
for w in range(2,5):
    g3.set_type(w, VertexType.Z)
g3.add_edges([(ZXi,i),(ZXo,o),(i,o),(i,gadget),(o,gadget)], EdgeType.HADAMARD)
g3.normalize()
replace_measurement(g3, gadget, "YZ", 0, [gadget])
highlight_missing_correction_set(g3)

In [17]:
g3 = Graph()
[ZXi1, ZXi2, ZXi3, ZXo1, ZXo2, ZXo3, i1, i2, i3, o1, o2, o3, v1, v2, v3, v4, gad1, gad2, gad3, gad4] = g3.add_vertices(20)
g3.set_inputs([ZXi1, ZXi2, ZXi3])
g3.set_outputs([ZXo1, ZXo2, ZXo3])
for w in range(6,20):
    g3.set_type(w, VertexType.Z)
for w in range(3):
    g3.add_edges([(w, w+6), (w+3, w+9)], EdgeType.HADAMARD)
g3.add_edges([(i1, v1),(v1,o1),(i2,v2),(v2,v3),(v3,o2),(i3,v4),(v4,o3)], EdgeType.HADAMARD)
for gad in [gad1, gad2, gad3, gad4]:
    g3.add_edges([(v2,gad),(v3,gad)], EdgeType.HADAMARD)
    replace_measurement(g3, gad, "YZ", 0, [gad])
g3.add_edges([(gad1,v1),(gad2,v1),(gad2,v4),(gad4,v4)], EdgeType.HADAMARD)
g3.normalize()
highlight_missing_correction_set(g3)

In [12]:
g4 = PhaseGadgetForm(qubits=4, phase_gadgets=7)\
   .set_all_connections_random(probability=0.5)\
   .set_many_connections(3, {0,1,2,3,4,5})\
   .to_graph()
g4.add_edges([(1,5),(1,9),(5,9)], EdgeType.HADAMARD)
highlight_missing_correction_set(g4)

In [13]:
g4 = Graph()
#i0 = add_vertex(g4, 0, -1, VertexType.Z, 0, ["I"])
i = add_vertex(g4, 0, 0, VertexType.Z, 0, ["I"])
u1 = add_vertex(g4, -1, 1, VertexType.Z, 0, [i])
u2 = add_vertex(g4,  1, 1, VertexType.Z, 0, [i])
w1 = add_vertex(g4, -1, 2, VertexType.Z, 0, [u1])
w2 = add_vertex(g4,  1, 2, VertexType.Z, 0, [u2])
add_vertex(g4, -1, 4, VertexType.Z, 0, [w1, "O"])
add_vertex(g4,  1, 4, VertexType.Z, 0, [w2, "O"])
replace_measurement(g4, w1, "YZ", 0, [w1])
replace_measurement(g4, w2, "YZ", 0, [w2])
zx.draw(g4)
highlight_missing_correction_set(g4)

# Notes
Dit proces kan niet een nul-rij introduceren. Wat je ook doet met de matrix eerder, je houdt minstens twee keer $1$ over. Als we dan in 1 index verschillen van het resultaat en je daar een nulrij mee kan kon maken, had je eerst een één-$1$-rij kunnen maken en extracten, wat tegenspraak is.

Any nul-rij in het process komt dus doordat de graaf waar je mee begon uberhaupt niet well-formed is.

EDIT: Het kan wel zo zijn dat vertices dieper in de graaf naar voren halen wel nulrijen kan maken.

# Todo
Definieer naast `gflow` ook een `glack` die aangeeft hoe vaak je bij de beste keuzes een ancilla moet toevoegen bij het pyzx-extractieprocess. (Natuurlijk hebben de standaard extractbare grafen een glack van 0.)

Hoe gedraagt zich dit? Is dit behouden onder `lcomp`s en `pivot`s? Hoe verandert dit onder spider nest identities als boven?

Doe ipv random grafen grafen met random measurement artifacts.